In [2]:
!pip install beautifulsoup4
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 33.3 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
import json  # Nếu cần xử lý dữ liệu JSON
import re
from urllib.parse import urljoin

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# mã cổ phiếu cần theo dõi tin tức
cp = 'VIC'

In [6]:
url_cafef = f'https://cafef.vn/du-lieu/tin-doanh-nghiep/{cp}/event.chn'
url_vneconomy = f'https://vneconomy.vn/tim-kiem.htm?q={cp}'

Crawl data cho CafeF

In [ ]:
#trong site tổng hợp tin tức lấy thông tin tên và ngày làm tên cho file txt:
#       thẻ divEvents >>> các thẻ <li> >>> thời gian <timeTitle>, tiêu đề <docnhanhTitle>, ấn vào link href
#trong site của bài báo lấy nội dung bên trong:
#       tất cả thẻ <p> trong thẻ <contentdetail>

In [20]:
def crawl_page(url, session, save_path):
    response = session.get(url)
    print(f"Response status code: {response.status_code}")
    soup = BeautifulSoup(response.content, 'html.parser')

    # Tìm thẻ div có id là divEvents
    div_events = soup.find('div', id='divEvents')
    print(f"div_events found: {div_events is not None}")

    # Tìm tất cả các thẻ <li> trong thẻ divEvents
    events = div_events.find_all('li')
    print(f"Number of events found: {len(events)}")

    for event in events:
        # Phần xử lý event giữ nguyên như cũ
        time_title = event.find('span', class_='timeTitle').get_text(strip=True)
        docnhanh_title_tag = event.find('a', class_='docnhanhTitle')
        docnhanh_title = docnhanh_title_tag['title']
        print(f"Event found: {time_title} - {docnhanh_title}")
        # đặt cấu trúc tên file
        file_name_not_final = re.sub(r"""[\\/*?:_,"<>'`‘’~|]""", ' ', f"{time_title}_{docnhanh_title}")
        file_name = file_name_not_final[:150].strip() + ".txt"

        href = docnhanh_title_tag['href']
        print(f"docnhanh_title_tag: {docnhanh_title_tag}")
        print(f"Article href: {href}")
        article_url = f"https://cafef.vn{href}"
        print(f"Article URL: {article_url}")

        article_response = session.get(article_url)
        print(f"Article response status code: {article_response.status_code}")

        if article_response.status_code != 200:
            print(f"Failed to retrieve article: {article_url}")
            continue

        article_soup = BeautifulSoup(article_response.content, 'html.parser')

        paragraphs_div = article_soup.find('div', class_='detail-content afcbc-body')
        if paragraphs_div is None:
            print(f"No paragraphs found in article: {article_url}")
            continue

        paragraphs = paragraphs_div.find_all('p')
        if not paragraphs:
            print(f"No <p> tags found in article: {article_url}")
            continue

        print(f"Number of paragraphs found: {len(paragraphs)}")

        with open(os.path.join(save_path, file_name), 'w', encoding='utf-8') as file:
            for paragraph in paragraphs:
                file.write(paragraph.get_text(strip=True) + '\n')
                print(f"Paragraph written: {paragraph.get_text(strip=True)}")

    # Tạo URL cho trang tiếp theo sử dụng đúng AJAX endpoint
    page_index = url.split('PageIndex=')[1].split('&')[0] if 'PageIndex=' in url else '1'
    next_page = int(page_index) + 1
    next_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex={next_page}&PageSize=30&Type=2"

    return next_url

# Thiết lập và chạy crawler
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw_cafef'
os.makedirs(save_path, exist_ok=True)

# URL ban đầu
initial_url = f"https://cafef.vn/du-lieu/Ajax/Events_RelatedNews_New.aspx?symbol={cp}&floorID=0&configID=0&PageIndex=1&PageSize=30&Type=2"

with requests.Session() as session:
    current_url = initial_url
    for page in range(1, 5):  # Crawl 10 trang, bạn có thể điều chỉnh số này
        print(f"\nCrawling page {page}")
        current_url = crawl_page(current_url, session, save_path)
        if not current_url:
            print("No more pages to crawl")
            break
        # Thêm delay giữa các request để tránh quá tải server
        time.sleep(2)


Crawling page 1
Response status code: 200
div_events found: True
Number of events found: 30
Event found: 15/01/2025 14:11 - Tỷ phú Phạm Nhật Vượng đã đầu tư những dự án nào tại Thanh Hóa?
docnhanh_title_tag: <a class="docnhanhTitle" href="/du-lieu/vic-2115508/ty-phu-pham-nhat-vuong-da-dau-tu-nhung-du-an-nao-tai-thanh-hoa.chn" title="Tỷ phú Phạm Nhật Vượng đã đầu tư những dự án nào tại Thanh Hóa?">Tỷ phú Phạm Nhật Vượng đã đầu tư những dự án nào tại Thanh Hóa?</a>
Article href: /du-lieu/vic-2115508/ty-phu-pham-nhat-vuong-da-dau-tu-nhung-du-an-nao-tai-thanh-hoa.chn
Article URL: https://cafef.vn/du-lieu/vic-2115508/ty-phu-pham-nhat-vuong-da-dau-tu-nhung-du-an-nao-tai-thanh-hoa.chn
Article response status code: 200
Number of paragraphs found: 21
Paragraph written: Ngày 15/1, tỷ phú Phạm Nhật Vượng cùng gia đình đã có mặt tại Thanh Hóa trong ngày trọng đại của con trai thứ hai là ông Phạm Nhật Minh Hoàng. Theo đó, gia đình người giàu nhất Việt Nam đã có mặt đầy đủ để tham dự lễ dãm ngọ giữ

Crawl data cho vn economy

In [ ]:
# tìm trong list báo về công ty đó
#        tìm trong <div> class = contentSearch  >>> tìm thẻ <div> có class = story_meta và thẻ <time> bên trong
#        tìm trong <div> class = contentSearch  >>>thẻ div class = story header sau đó tìm trong thẻ <a> bên trong sẽ có các thông tin bao gồm: href có link để crawl dữ liệu, và  title là tên bài viết
# tìm trong link href >>> class = detail_content >>> tất cả các thẻ <p>

In [29]:
import requests
import os
from bs4 import BeautifulSoup

# Setup
search_url = 'https://vneconomy.vn/'
save_path = '/content/drive/MyDrive/NCKH-2025/data_raw_vneconomy'
os.makedirs(save_path, exist_ok=True)

# Function to save article content to a file
def save_article(title, published_date, url_article, session):
    article_url = search_url + url_article
    print(f"Fetching article: {article_url}")  # Debug: Print URL being fetched
    response = session.get(article_url)
    print(f"Status code: {response.status_code}")  # Debug: Print status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        detail_content = soup.find('div', class_='detail__content')
        if detail_content:
            paragraphs = detail_content.find_all('p')
            if paragraphs:
                # Tạo tên file an toàn
                safe_title = "".join(c if c.isalnum() or c in [' ', '-', '_'] else '_' for c in title)
                filename = f"{published_date} {safe_title[:100]}.txt"
                filepath = os.path.join(save_path, filename)
                try:
                    with open(filepath, 'w', encoding='utf-8') as file:
                        for paragraph in paragraphs:
                            text = paragraph.get_text(strip=True)
                            if text:
                                file.write(text + '\n\n')
                                print(f"Paragraph written: {text}")  # Debug: Print each paragraph text
                    print(f"Saved: {filename}")
                except Exception as e:
                    print(f"Failed to save file {filename}: {e}")
            else:
                print(f"No paragraphs found in article: {article_url}")
        else:
            print(f"No detail content found in article: {article_url}")
    else:
        print(f"Failed to retrieve article: {article_url}")

# Main function to crawl data
def crawl_data(base_url, num_pages):
    session = requests.Session()
    for page in range(1, num_pages + 1):
        url = base_url + str(page)
        print(f"Fetching page: {url}")  # Debug: Print URL being fetched
        response = session.get(url)
        print(f"Status code: {response.status_code}")  # Debug: Print status code
        if response.status_code == 200:
            try:
                data = response.json().get('List', [])
                for item in data:
                    print(f"Item: {item}")  # Debug: Print each item data
                    title = item.get('Title')
                    published_date = item.get('PublishedDate')
                    url_article = item.get('UrlArticle')
                    if title and published_date and url_article:
                        save_article(title, published_date, url_article, session)
            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON from {url}: {e}")
        else:
            print(f"Failed to retrieve data from: {url}")

# Start crawling
num_pages = 10  # Số lần lặp lại quá trình crawl
base_url = f'https://search.hemera.com.vn/search/1/VIC/time/'  # Thay đổi mã CP theo nhu cầu
crawl_data(base_url, num_pages)

Fetching page: https://search.hemera.com.vn/search/1/VIC/time/1
Status code: 200
Item: {'Id': '1_638696546671771044', 'Title': 'Dự án <em>VIC</em> Grand Square được vinh danh tại giải thưởng bất động sản Đông Nam Á', 'JsonTag': '[{"Name":"Dự án VIC Grand Square","Url":"du-an-vic-grand-square"},{"Name":"Vneconomy","Url":"vneconomy"}]', 'PublishedDate': '2024-12-13T03:12:58.703Z', 'CatName': 'Bất động sản', 'UrlCate': 'dia-oc', 'Sapo': ' sạn Four Seasons Hotel Bangkok, Thái Lan dự án <em>VIC</em> Grand Square của chủ đầu tư <em>VIC</em> Group đã bất ngờ', 'Avatar': '2024/12/13/z6125163269366-46a397203c4fa7d3fb94f6c2091f739c.jpg', 'UrlArticle': 'du-an-vic-grand-square-duoc-vinh-danh-tai-giai-thuong-bat-dong-san-dong-nam-a', 'NameAuthor': 'Khánh  Huyền'}
Fetching article: https://vneconomy.vn/du-an-vic-grand-square-duoc-vinh-danh-tai-giai-thuong-bat-dong-san-dong-nam-a
Status code: 200
Paragraph written: Giải thưởng Dot Property Asia Awards là một trong những giải thưởng bất động sản uy tí